In [62]:
#installed TKinter to stop the GUI errors for the figures
#https://docs.python.org/3/library/tkinter.html

from sqlalchemy import create_engine
import tkinter
import matplotlib
matplotlib.use('TkAgg')

import pymysql
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt


In [2]:
credit = pd.read_csv('credit.csv')

In [3]:
credit = credit.drop_duplicates()

In [4]:
#X1: Amount of the given credit
#X2: Gender (1 = male; 2 = female)
#X3: Education (1 = graduate school; 2 = university; 3 = high school; 0, 4, 5, 6 = others)
#X4: Marital status (1 = married; 2 = single; 3 = divorce; 0=others).
#X5: Age (year)
#X6 - X11: History of past payment.
#X12-X17: Amount of bill statement (NT dollar).
#X18-X23: Amount of previous payment (NT dollar)
#Y: client's behavior

#for payment history
#-2: No consumption; -1: Paid in full; 0: The use of revolving credit; 1 = payment delay
#for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight
#months; 9 = payment delay for nine months and above.

credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30002 entries, 0 to 30203
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MyUnknownColumn  30001 non-null  object
 1   X1               30002 non-null  object
 2   X2               30002 non-null  object
 3   X3               30002 non-null  object
 4   X4               30002 non-null  object
 5   X5               30002 non-null  object
 6   X6               30002 non-null  object
 7   X7               30002 non-null  object
 8   X8               30002 non-null  object
 9   X9               30002 non-null  object
 10  X10              30002 non-null  object
 11  X11              30002 non-null  object
 12  X12              30002 non-null  object
 13  X13              30002 non-null  object
 14  X14              30002 non-null  object
 15  X15              30002 non-null  object
 16  X16              30002 non-null  object
 17  X17              30002 non-null

In [5]:
credit.head()

,MyUnknownColumn,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,female,university,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,default
2,2,120000,female,university,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
3,3,90000,female,university,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
4,4,50000,female,university,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default


In [6]:
credit = credit.replace(['','null'],[np.nan, np.nan])

In [7]:
#checking for null data
print(credit.isnull().sum())

MyUnknownColumn    1
X1                 0
X2                 0
X3                 0
X4                 0
X5                 0
X6                 0
X7                 0
X8                 0
X9                 0
X10                0
X11                0
X12                0
X13                0
X14                0
X15                0
X16                0
X17                0
X18                0
X19                0
X20                0
X21                0
X22                0
X23                0
Y                  0
dtype: int64


In [8]:
#finding the null data
null_series = pd.isnull(credit["MyUnknownColumn"])

In [9]:
credit[null_series]

,MyUnknownColumn,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
202,NaN,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y


In [10]:
#removing null found
credit.drop(credit.index[202], inplace = True)

In [11]:
print(credit.isnull().sum())

MyUnknownColumn    0
X1                 0
X2                 0
X3                 0
X4                 0
X5                 0
X6                 0
X7                 0
X8                 0
X9                 0
X10                0
X11                0
X12                0
X13                0
X14                0
X15                0
X16                0
X17                0
X18                0
X19                0
X20                0
X21                0
X22                0
X23                0
Y                  0
dtype: int64


In [12]:
#from the presentation to change dtypes to int without a lot of coding
credit.to_csv('cleancredit.csv', index = False)

In [13]:
#header =1 to drop the first row 
cleancredit= pd.read_csv('cleancredit.csv', header = 1)

In [14]:
cleancredit.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,female,university,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,default
1,2,120000,female,university,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
2,3,90000,female,university,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
3,4,50000,female,university,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default
4,5,50000,male,university,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,not default


In [15]:
cleancredit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  int64 
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  object
 3   EDUCATION                   30000 non-null  object
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_0                       30000 non-null  int64 
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [16]:
#changing object type items to int 
cleancredit = pd.get_dummies(cleancredit)

In [17]:
cleancredit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   ID                                      30000 non-null  int64
 1   LIMIT_BAL                               30000 non-null  int64
 2   MARRIAGE                                30000 non-null  int64
 3   AGE                                     30000 non-null  int64
 4   PAY_0                                   30000 non-null  int64
 5   PAY_2                                   30000 non-null  int64
 6   PAY_3                                   30000 non-null  int64
 7   PAY_4                                   30000 non-null  int64
 8   PAY_5                                   30000 non-null  int64
 9   PAY_6                                   30000 non-null  int64
 10  BILL_AMT1                               30000 non-null  int64
 11  BILL_AMT2      

In [18]:
header = cleancredit.dtypes.index
print(header)

Index(['ID', 'LIMIT_BAL', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
       'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
       'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
       'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SEX_female',
       'SEX_male', 'EDUCATION_graduate school', 'EDUCATION_high school',
       'EDUCATION_other', 'EDUCATION_university',
       'default payment next month_default',
       'default payment next month_not default'],
      dtype='object')


In [63]:
plt.hist(cleancredit['LIMIT_BAL'])
plt.show()

In [64]:
plt.hist(cleancredit['LIMIT_BAL'], bins=4)
plt.show()

In [65]:
plt.plot(cleancredit['LIMIT_BAL'])
plt.show()

In [66]:
x = cleancredit['PAY_0']
y = cleancredit['PAY_2']

In [67]:
plt.scatter(x,y)
plt.show()

In [55]:
header = cleancredit.dtypes.index
print(header)

Index(['ID', 'LIMIT_BAL', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
       'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
       'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
       'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SEX_female',
       'SEX_male', 'EDUCATION_graduate school', 'EDUCATION_high school',
       'EDUCATION_other', 'EDUCATION_university',
       'default payment next month_default',
       'default payment next month_not default'],
      dtype='object')


In [68]:
A = cleancredit['BILL_AMT1']
plt.boxplot(A,0,'gD')
plt.show()

In [57]:
#ID,education,sex,marriage,age not really correlated to anything?
#nothing really correlates to default payment
corrMat = cleancredit.corr()
print(corrMat)

                                              ID  LIMIT_BAL  MARRIAGE  \
ID                                      1.000000   0.026179 -0.029079   
LIMIT_BAL                               0.026179   1.000000 -0.108139   
MARRIAGE                               -0.029079  -0.108139  1.000000   
AGE                                     0.018678   0.144713 -0.414170   
PAY_0                                  -0.030575  -0.271214  0.019917   
PAY_2                                  -0.011215  -0.296382  0.024199   
PAY_3                                  -0.018494  -0.286123  0.032688   
PAY_4                                  -0.002735  -0.267460  0.033122   
PAY_5                                  -0.022199  -0.249411  0.035629   
PAY_6                                  -0.020270  -0.235195  0.034345   
BILL_AMT1                               0.019389   0.285430 -0.023472   
BILL_AMT2                               0.017982   0.278314 -0.021602   
BILL_AMT3                               0.024354   

In [58]:
covMat = cleancredit.cov()
print(covMat)

                                                  ID     LIMIT_BAL  \
ID                                      7.500250e+07  2.941664e+07   
LIMIT_BAL                               2.941664e+07  1.683446e+10   
MARRIAGE                               -1.314525e+02 -7.323670e+03   
AGE                                     1.491057e+03  1.730767e+05   
PAY_0                                  -2.975726e+02 -3.954593e+04   
PAY_2                                  -1.162770e+02 -4.603765e+04   
PAY_3                                  -1.916922e+02 -4.443225e+04   
PAY_4                                  -2.769132e+01 -4.057181e+04   
PAY_5                                  -2.178600e+02 -3.667056e+04   
PAY_6                                  -2.018774e+02 -3.509308e+04   
BILL_AMT1                               1.236447e+07  2.727020e+09   
BILL_AMT2                               1.108392e+07  2.570130e+09   
BILL_AMT3                               1.462688e+07  2.548533e+09   
BILL_AMT4           

In [69]:
plt.hist(cleancredit['default payment next month_default'])
plt.show()

In [60]:
 cleancredit.head()

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT5,PAY_AMT6,SEX_female,SEX_male,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_other,EDUCATION_university,default payment next month_default,default payment next month_not default
0,1,20000,1,24,2,2,-1,-1,-2,-2,...,0,0,1,0,0,0,0,1,1,0
1,2,120000,2,26,-1,2,0,0,0,2,...,0,2000,1,0,0,0,0,1,1,0
2,3,90000,2,34,0,0,0,0,0,0,...,1000,5000,1,0,0,0,0,1,0,1
3,4,50000,1,37,0,0,0,0,0,0,...,1069,1000,1,0,0,0,0,1,0,1
4,5,50000,1,57,-1,0,-1,0,0,0,...,689,679,0,1,0,0,0,1,0,1


In [61]:
plt.plot(cleancredit['default payment next month_default'])
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\2137818266.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [31]:
x1 = cleancredit['PAY_2']
y1 = cleancredit['PAY_3']
plt.scatter(x1,y1)
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\163614434.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [32]:
x2 = cleancredit['PAY_3']
y2 = cleancredit['PAY_4']
plt.scatter(x2,y2)
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\2537304620.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [33]:
#dropping some columns to have less distractions
credit2= cleancredit.copy()

In [34]:
credit2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   ID                                      30000 non-null  int64
 1   LIMIT_BAL                               30000 non-null  int64
 2   MARRIAGE                                30000 non-null  int64
 3   AGE                                     30000 non-null  int64
 4   PAY_0                                   30000 non-null  int64
 5   PAY_2                                   30000 non-null  int64
 6   PAY_3                                   30000 non-null  int64
 7   PAY_4                                   30000 non-null  int64
 8   PAY_5                                   30000 non-null  int64
 9   PAY_6                                   30000 non-null  int64
 10  BILL_AMT1                               30000 non-null  int64
 11  BILL_AMT2      

In [35]:
credit2.drop(columns=['EDUCATION_graduate school','EDUCATION_high school','EDUCATION_other',
                     'EDUCATION_university','SEX_male','SEX_female','AGE','MARRIAGE'], inplace =True, axis =1)

In [36]:
cleancredit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   ID                                      30000 non-null  int64
 1   LIMIT_BAL                               30000 non-null  int64
 2   MARRIAGE                                30000 non-null  int64
 3   AGE                                     30000 non-null  int64
 4   PAY_0                                   30000 non-null  int64
 5   PAY_2                                   30000 non-null  int64
 6   PAY_3                                   30000 non-null  int64
 7   PAY_4                                   30000 non-null  int64
 8   PAY_5                                   30000 non-null  int64
 9   PAY_6                                   30000 non-null  int64
 10  BILL_AMT1                               30000 non-null  int64
 11  BILL_AMT2      

In [37]:
credit2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 22 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   ID                                      30000 non-null  int64
 1   LIMIT_BAL                               30000 non-null  int64
 2   PAY_0                                   30000 non-null  int64
 3   PAY_2                                   30000 non-null  int64
 4   PAY_3                                   30000 non-null  int64
 5   PAY_4                                   30000 non-null  int64
 6   PAY_5                                   30000 non-null  int64
 7   PAY_6                                   30000 non-null  int64
 8   BILL_AMT1                               30000 non-null  int64
 9   BILL_AMT2                               30000 non-null  int64
 10  BILL_AMT3                               30000 non-null  int64
 11  BILL_AMT4      

In [38]:
corrMat = credit2.corr()
print(corrMat)

                                              ID  LIMIT_BAL     PAY_0  \
ID                                      1.000000   0.026179 -0.030575   
LIMIT_BAL                               0.026179   1.000000 -0.271214   
PAY_0                                  -0.030575  -0.271214  1.000000   
PAY_2                                  -0.011215  -0.296382  0.672164   
PAY_3                                  -0.018494  -0.286123  0.574245   
PAY_4                                  -0.002735  -0.267460  0.538841   
PAY_5                                  -0.022199  -0.249411  0.509426   
PAY_6                                  -0.020270  -0.235195  0.474553   
BILL_AMT1                               0.019389   0.285430  0.187068   
BILL_AMT2                               0.017982   0.278314  0.189859   
BILL_AMT3                               0.024354   0.283236  0.179785   
BILL_AMT4                               0.040351   0.293988  0.179125   
BILL_AMT5                               0.016705   

In [39]:
credit2.head()

,ID,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month_default,default payment next month_not default
0,1,20000,2,2,-1,-1,-2,-2,3913,3102,...,0,0,0,689,0,0,0,0,1,0
1,2,120000,-1,2,0,0,0,2,2682,1725,...,3455,3261,0,1000,1000,1000,0,2000,1,0
2,3,90000,0,0,0,0,0,0,29239,14027,...,14948,15549,1518,1500,1000,1000,1000,5000,0,1
3,4,50000,0,0,0,0,0,0,46990,48233,...,28959,29547,2000,2019,1200,1100,1069,1000,0,1
4,5,50000,-1,0,-1,0,0,0,8617,5670,...,19146,19131,2000,36681,10000,9000,689,679,0,1


In [40]:
plt.hist(credit2['PAY_0'])
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\989709769.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [41]:
A = credit2['PAY_0']
plt.boxplot(A,0,'gD')
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\1911838248.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [42]:
A = credit2['PAY_AMT1']
plt.boxplot(A,0,'gD')
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\2686984835.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [43]:
plt.plot(credit2['PAY_0'])
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\1479470176.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [44]:
x3 = cleancredit['LIMIT_BAL']
y3 = cleancredit['default payment next month_default']
plt.scatter(x3,y3)
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\1061449895.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [45]:
#switch x and y
x4 = cleancredit['LIMIT_BAL']
y4 = cleancredit['default payment next month_default']
plt.scatter(y4,x4)
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\2799164355.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [46]:
B = cleancredit['LIMIT_BAL']
plt.boxplot(B,0,'gD')
plt.show()

C:\Users\alden\AppData\Local\Temp\ipykernel_21496\1163564633.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [79]:
x5 = cleancredit['PAY_0']
y5 = cleancredit['default payment next month_default']
plt.scatter(x5,y5)
plt.show()

In [71]:
#payment amounts closer to 40k (maybe 30k?) and higher did not default? 
x6 = cleancredit['PAY_AMT1']
y6 = cleancredit['default payment next month_default']
plt.scatter(x6,y6)
plt.show()

In [74]:
#higher payments amounts still did not  default
x61 = cleancredit['PAY_AMT2']
y61 = cleancredit['default payment next month_default']
plt.scatter(x61,y61)
plt.show()

In [77]:
#some defaults at the higher amounts but not as many as the lower amounts
x62 = cleancredit['PAY_AMT3']
y62 = cleancredit['default payment next month_default']
plt.scatter(x62,y62)
plt.show()

In [70]:
#extreme low and high bill amounts did not default?
x7 = cleancredit['BILL_AMT1']
y7 = cleancredit['default payment next month_default']
plt.scatter(x7,y7)
plt.show()

In [82]:
plt.hist(credit2['PAY_0'])
plt.hist(credit2['PAY_2'])
plt.hist(credit2['PAY_3'])
plt.hist(credit2['PAY_4'])
plt.hist(credit2['PAY_5'])
plt.hist(credit2['PAY_6'])
plt.show()